## Product metrics

In [4]:
import pandas as pd
import numpy as np 
from datetime import timedelta
from datetime import datetime

import requests
from urllib.parse import urlencode

In [3]:
# Create link to a file
case_data_url = 'https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q'

In [7]:
# Create a function for a request to Yandex:
def download_yandex(public_key):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url

In [8]:
# Loading data
df = pd.read_csv(download_yandex(case_data_url), parse_dates=['date'], dayfirst=True)
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


#### 1. MAU February:

In [9]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [10]:
# Create a column with month
df['month'] = df.date.dt.month

In [11]:
# MAU February:
df.query('month == 2').device_id.nunique()

75032

#### 2. Number of installs in January:

In [12]:
df.query('month == 1 & event == "app_install"').device_id.count()

80297

#### 3. Assign cohorts to users by the day the application was installed and calculate the conversion from installation to purchase for them within 7 days. Which cohort had the highest conversion rate?

Note: Consider a user to have converted if no more than 7 days have passed from the moment of installation to the first purchase.

In [20]:
# Creating data with the number of app installs each days 
df_inst = df.query('event == "app_install"') \
    .groupby('date') \
    .agg({'device_id' : 'count'})

In [22]:
# Creating data with the first date of app installs 
df_install = df.query('event == "app_install"').groupby('device_id').agg({'date' : 'min'})

In [23]:
# Creating data with the first purchase
df_purchase = df.query('event == "purchase"').groupby('device_id').agg({'date' : 'min'})

In [24]:
# Creating column with difference between app install date and first purchase date
df_install['diff'] = df_purchase.date - df_install.date

In [27]:
# Formating different date
df_install['diff'] = df_install['diff'].dt.days
df_install = df_install.dropna().reset_index()

In [29]:
# Creating data with the number of purchases in 7 days 
df_purch_7d = df_install.query('diff<=7').groupby('date').agg({'device_id' : 'count'})

In [41]:
# Highest conversion rate to purchase after installation on day 7
CR_7d = df_purch_7d / df_inst
CR_7d.sort_values('device_id', ascending = False).reset_index().rename(columns = {'device_id':'CR_7d'}).head(1)

,date,CR_7d
0,2020-01-01,0.393406


#### 4. Which paid marketing channel brought in the most new users?

In [39]:
df.query('event == "app_install" & utm_source != "-"') \
    .groupby('utm_source', as_index=False) \
    .agg({'device_id' : 'count'}) \
    .rename(columns = {'device_id':'users'}) \
    .sort_values('users', ascending = False).head(1)

,utm_source,users
5,yandex-direct,29368


#### 5. Users who came from which channels showed the lowest conversion on their first purchase?

In [42]:
# Number of users who made their first purchase
df_first_purch = df.query('event == "purchase"') \
    .groupby(['utm_source', 'device_id'], as_index = False) \
    .agg({'date' : 'min'}) \
    .groupby('utm_source') \
    .agg({'device_id' : 'count'}) \
    .sort_values('device_id', ascending = False)

In [43]:
# Number of users who opened the app for the first time
df_start_app = df.query('event == "app_start"') \
    .groupby(['utm_source', 'device_id'], as_index = False) \
    .agg({'date' : 'min'}) \
    .groupby('utm_source') \
    .agg({'device_id' : 'count'}) \
    .sort_values('device_id', ascending = False)

In [44]:
# CR - first purchase conversion
(df_first_purch / df_start_app).sort_values('device_id').head(1)

,device_id
utm_source,
yandex-direct,0.295477


#### 6. Users who came from which channel have a higher median first check? (only the first purchases of users are taken into account)

In [45]:
# first - first purchase choice
df.query('event == "purchase" & utm_source != "-"') \
    .groupby(['utm_source', 'device_id'], as_index = False) \
    .agg({'date' :'min', 'purchase_sum' : 'first'}) \
     .groupby('utm_source') \
    .agg({'purchase_sum' : 'median'}) \
    .sort_values('purchase_sum', ascending = False).head(1)

,purchase_sum
utm_source,
referal,395.0


#### 7. Which paid acquisition channel (among ads) has the highest ROMI?

In [541]:
df.groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}) \
    .sort_values('purchase_sum', ascending = False)

,utm_source,purchase_sum
0,-,21449749.5
5,vk_ads,16389652.5
3,instagram_ads,14546969.0
6,yandex-direct,13915368.0
2,google_ads,12868276.0
1,facebook_ads,12249901.0
4,referal,8837044.5


In [46]:
# ROMI Yandex-direct
(df.query('utm_source == "yandex-direct"') \
    .groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}).purchase_sum - 10491707) / 10491707

0    0.326321
Name: purchase_sum, dtype: float64

In [47]:
# ROMI Google_ads
(df.query('utm_source == "google_ads"') \
    .groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}).purchase_sum - 10534878) / 10534878

0    0.221493
Name: purchase_sum, dtype: float64

In [48]:
# ROMI Facebook_ads
(df.query('utm_source == "facebook_ads"') \
    .groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}).purchase_sum - 8590498) / 8590498

0    0.425983
Name: purchase_sum, dtype: float64

In [49]:
# ROMI Instagram_ads
(df.query('utm_source == "instagram_ads"') \
    .groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}).purchase_sum - 8561626) / 8561626

0    0.69909
Name: purchase_sum, dtype: float64

In [50]:
# ROMI Vk_ads
(df.query('utm_source == "vk_ads"') \
    .groupby(['utm_source'], as_index = False) \
    .agg({'purchase_sum' : 'sum'}).purchase_sum - 9553531) / 9553531

0    0.71556
Name: purchase_sum, dtype: float64

#### VK has the highest ROMI (0.71556)